Zadania
Zadanie 1
Wczytaj plik zamowienia.csv do ramki pandas, a następnie w kilku miejscach (ale nie w pierwszych 10 wierszach) wstaw wartość NaN, aby zasymulować wartości brakujące. Zapisz ramkę do pliku zamowienia_missing.csv. Wczytaj teraz plik do ramki Dask i sprawdź jakie typy danych zostały przydzielone. Czy zgadzają się z typami z oryginalnego pliku? Wykonaj dowolne obliczenia na całej ramce Dask, aby wymusić wywołanie .compute(). Czy pojawił się błąd dotyczący niespójności typów danych? Spróbuj uruchomić kilka razy funkcję wczytywania danych do ramki Dask dataframe z różnymi wartościami parametru samples. Dokumentacja dask.dataframe.read_csv(): https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html

Zadanie 2
Ze strony https://docs.dask.org/en/stable/dashboard.html skonfiguruj plugin Dask dashboard dla Jupyter Lab i przetestuj jego działanie.

Zadanie 3
Skonfiguruj lokalny klaster (Client) tak, aby nie zaalokował wszystkich zasobów (np. zostaw 8 GB RAM dla systemu hosta + 2 rdzenie). Pobierz dane udostępnione na poprzednich zajęciach (https://huggingface.co/datasets/vargr/private_instagram/tree/refs%2Fconvert%2Fparquet/default/train) i załaduj do ramki Dask tyle części ile zdołasz w formie bez optymalizacji. Zmierz czas tej operacji.

Zadanie 4
Wykonaj kilka operacji na klastrze lokalnym z danymi z zadania 3:

wyświetl top 10 użytkowników z najwyższą liczbą like'ów,
pobierz dane tylko za pierwsze półrocze 2019 roku. Każdorazowo zmierz i wyświetl czas operacji i obserwuj dashboard.

Zadanie 5
Wczytaj te same dane do ramki Dask co w zadaniu 3, ale podaj typy danych, które zostały wybrane w procesie optymalizacji wykonanej w zadaniach z lab 01. Porównaj czas ładowania z zadaniem 3. Wykonaj również te same operacje co w zadaniu 4 i porównaj czas. Śledź wykonanie zadań patrząć na graf wywołań.

Zadanie 6
Podziel tablicę darr z przykładów na inne liczby chunków (eksperymentuj) i wykonaj te same obliczenie (średnia). Dla każdej liczby chunków wypisz czas obliczeń (wykonaj to samo obliczenie minimum 10 razy, aby nieco uwiarygodnić wyniki i uśrednij) i porównaj wyniki. Napisz wniosek o wynikach swoich eksperymentów i automatycznego podziału na chunki. Czy udało Ci się osiągnąć lepszą wydajność niż przy domyślnych ustawieniach?

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client
import time

In [2]:
df = pd.read_csv('zamowienia.csv')
np.random.seed(42)
num_nan = 5
cols = df.columns.tolist()

for _ in range(num_nan):
    row_idx = np.random.randint(10, len(df))
    col_idx = np.random.choice(cols) 
    df.at[row_idx, col_idx] = np.nan 
    
df.to_csv('zamowienia_missing.csv', index=False)
print("Plik zamowienia_missing.csv został zapisany ")

Plik zamowienia_missing.csv został zapisany 


In [3]:
dask_df = dd.read_csv('zamowienia_missing.csv')
print("Typy danych w ramce Dask:")
print(dask_df.dtypes)

original_df = pd.read_csv('zamowienia.csv')
print("\nTypy danych w oryginalnym pliku pandas:")
print(original_df.dtypes)


Typy danych w ramce Dask:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
dtype: object

Typy danych w oryginalnym pliku pandas:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    object
dtype: object


In [4]:
numeric_columns = dask_df.select_dtypes(include=['number'])
sum_result = numeric_columns.sum().compute()
print("\nWynik obliczeń (sumowanie wartości dla kolumn numerycznych):")
print(sum_result)


Wynik obliczeń (sumowanie wartości dla kolumn numerycznych):
Series([], dtype: float64)


In [5]:
for sample_size in [100_000, 1_000_000, 10_000_000]:
    print(f"\nWczytywanie danych z sample={sample_size} bajtów:")
    dask_df_sampled = dd.read_csv('zamowienia_missing.csv', sample=sample_size)
    print(dask_df_sampled.dtypes)


Wczytywanie danych z sample=100000 bajtów:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
dtype: object

Wczytywanie danych z sample=1000000 bajtów:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
dtype: object

Wczytywanie danych z sample=10000000 bajtów:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
dtype: object


Zadanie 2

In [6]:
# from dask.distributed import Client
# 
# client = Client()
# client

In [7]:
arr = da.random.random((10000, 10000), chunks=(1000, 1000))
result = arr.sum().compute()

Zadanie 3

In [8]:
client = Client(memory_limit='8GB', n_workers=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 12,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54828,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:54843,Total threads: 6
Dashboard: http://127.0.0.1:54844/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:54831,


In [9]:
parquet_files = "path_to_downloaded_files/*.parquet"
dask_df = dd.read_parquet(parquet_files)
print(dask_df.info())
%time dask_df.compute()

<class 'dask.dataframe.core.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None
CPU times: total: 0 ns
Wall time: 11.3 ms


""


In [10]:
start_time = time.time()
dask_df.compute()
end_time = time.time()
print(f"Czas wczytywania danych: {end_time - start_time:.2f} sekundy")

Czas wczytywania danych: 0.02 sekundy


Zadanie 4